In [ ]:
#This script searches for new subs, copies raw T2w from processed iBEAT folder and raw qT1 from subject folder, and generates a binary mask. 
#Be sure to cheange filepaths 

In [7]:
from nipype.interfaces import freesurfer as fs
import pandas as pd
import os
import glob
from shutil import copyfile


In [24]:
crawfp = '/Volumes/iang/active/BABIES/BABIES_Crossectional/BABIES_Crossectional-T1'
lrawfp = '/Volumes/iang/active/BABIES/BABIES_Longitudinal/BABIES_Longitudinal-T3'
newfp= '/Volumes/iang/active/BABIES/BABIES_qT1/newproc'
ibeatdir = '/Volumes/iang/active/BABIES/BABIES_ibeat/processed_brains'

In [80]:
def getsubs_c():
    c_sub = pd.Series(glob.glob(crawfp+'/'+'*-C-T1'))
    c_subs = c_sub.apply(lambda x: os.path.splitext(os.path.basename(x))[0])
    c_sub = c_subs.str.split('-',expand=True)
    c_subs = c_sub[0].tolist()
    return c_subs

def getsubs_l():
    l_sub = pd.Series(glob.glob(lrawfp+'/'+'*-L-T3'))
    l_subs = l_sub.apply(lambda x: os.path.splitext(os.path.basename(x))[0])
    l_sub = l_subs.str.split('-',expand=True)
    l_subs = l_sub[0].tolist()
    return l_subs
getsubs_l()

['1036',
 '1027',
 '1023',
 '1033',
 '1078',
 '1010',
 '1029',
 '1005',
 '1024',
 '1020',
 '1011']

In [81]:
#mri_binarize --i 100-skullstripped_anat.nii --o binarized.nii --min 0 --max 1 --inv
csubs = getsubs_c()
lsubs = getsubs_l()

for sub in csubs:
    ibraw = ibeatdir+'/BABIES_C/processed_T2w'+'/'+sub+'-skullstripped_anat.nii'
    subdir = newfp + '/'+ sub
    ibcopy = subdir+'/'+sub+'_skullstripped.nii'
    mask = subdir+'/'+sub+'-binarized_mask.nii'
    qt1raw_pe1 = crawfp +'/'+sub+'-C-T1/anat/qt1/recon/qt1_pe1.nii.gz'
    qt1raw_pe0 = crawfp +'/'+sub+'-C-T1/anat/qt1/recon/qt1_pe0.nii.gz'
    if os.path.exists(ibraw):
        print('iBEAT T2w exists for {}'.format(sub))
        if os.path.exists(subdir):
            print('Subject Directory exists for {}'.format(sub))
        else:
            if os.path.exists(qt1raw_pe1):
                if os.path.exists(qt1raw_pe0):
                    print('All necessary data exists for {}, making subject directory'.format(sub))
                    os.makedirs(subdir)
                else:
                    print('qT1 pe0 not found for {}'.format(sub))
            else:
                print('qT1 pe1 not found for {}'.format(sub))
    else:
        print('Processed T2w does not exist for {}'.format(sub))
    if os.path.exists(ibraw):
        if os.path.exists(ibcopy):
            print('T2w already copied over for {}'.format(sub))
        else:
            if os.path.exists(subdir):
                print('Copying over T2w for {}'.format(sub))
                copyfile(ibraw, ibcopy)
            else:
                print('Subject directory not created for {} because subject is missing data'.format(sub))
    else:
        print('Processed T2w not found for {}'.format(sub))
    if os.path.exists(ibcopy):
        if os.path.exists(newfp+'/'+sub+'/'+ sub+'-binarized_mask.nii'):
                          print('*****Mask for {} already exists*****'.format(sub))
        else:
                          print('Computing Mask for {}'.format(sub))
                          binvol = fs.Binarize(in_file = ibcopy, 
                                               min=0,
                                               max = 1,
                                               invert = True,
                                               binary_file= mask)
                          binvol.run()
    else:
              print('Skullstripped T2w does not exist for {}'.format(sub))
              
            
#     binvol = fs.Binarize(in_file = prac + '/100-skullstripped_anat.nii', 
#                       min=0,
#                       max = 1,
#                       invert = True,
#                       binary_file= prac + '/binarized_1.nii')
#     binvol.run()

Processed T2w does not exist for 033x
Processed T2w not found for 033x
Skullstripped T2w does not exist for 033x
iBEAT T2w exists for 056
qT1 pe1 not found for 056
Subject directory not created for 056 because subject is missing data
Skullstripped T2w does not exist for 056
iBEAT T2w exists for 067
Subject Directory exists for 067
T2w already copied over for 067
*****Mask for 067 already exists*****
Processed T2w does not exist for 020
Processed T2w not found for 020
Skullstripped T2w does not exist for 020
Processed T2w does not exist for 028x
Processed T2w not found for 028x
Skullstripped T2w does not exist for 028x
iBEAT T2w exists for 122
qT1 pe0 not found for 122
Subject directory not created for 122 because subject is missing data
Skullstripped T2w does not exist for 122
Processed T2w does not exist for 021
Processed T2w not found for 021
Skullstripped T2w does not exist for 021
Processed T2w does not exist for 010
Processed T2w not found for 010
Skullstripped T2w does not exist 

In [37]:
lsubs = getsubs_l()
for sub in lsubs:
    ibraw = ibeatdir+'/BABIES_LT3/processed_T2w'+'/'+sub+'-skullstripped_anat.nii'
    subdir = newfp + '/'+ sub
    ibcopy = subdir+'/'+sub+'_skullstripped.nii'
    mask = subdir+'/'+sub+'-binarized_mask.nii'
    qt1raw_pe1 = lrawfp +'/'+sub+'-L-T3/anat/qt1/recon/qt1_pe1.nii.gz'
    qt1raw_pe0 = lrawfp +'/'+sub+'-L-T3/anat/qt1/recon/qt1_pe0.nii.gz'
    if os.path.exists(ibraw):
        print('iBEAT T2w exists for {}'.format(sub))
        if os.path.exists(subdir):
            print('Subject Directory exists for {}'.format(sub))
        else:
            if os.path.exists(qt1raw_pe1):
                if os.path.exists(qt1raw_pe0):
                    print('All necessary data exists for {}, making subject directory'.format(sub))
                    os.makedirs(subdir)
                else:
                    print('qT1 pe0 not found for {}'.format(sub))
            else:
                print('qT1 pe1 not found for {}'.format(sub))
    else:
        print('Processed T2w does not exist for {}'.format(sub))
    if os.path.exists(ibraw):
        if os.path.exists(ibcopy):
            print('T2w already copied over for {}'.format(sub))
        else:
            if os.path.exists(subdir):
                print('Copying over T2w for {}'.format(sub))
                copyfile(ibraw, ibcopy)
            else:
                print('Subject directory not created for {} because subject is missing data'.format(sub))
    else:
        print('Processed T2w not found for {}'.format(sub))
    if os.path.exists(ibcopy):
        if os.path.exists(newfp+'/'+sub+'/'+ sub+'-binarized_mask.nii'):
                          print('*****Mask for {} already exists*****'.format(sub))
        else:
                          print('Computing Mask for {}'.format(sub))
                          binvol = fs.Binarize(in_file = ibcopy, 
                                               min=0,
                                               max = 1,
                                               invert = True,
                                               binary_file= mask)
                          binvol.run()
    else:
              print('Skullstripped T2w does not exist for {}'.format(sub))
              

iBEAT T2w exists for 1036
Subject Directory exists for 1036
T2w already copied over for 1036
*****Mask for 1036 already exists*****
iBEAT T2w exists for 1027
Subject Directory exists for 1027
T2w already copied over for 1027
*****Mask for 1027 already exists*****
iBEAT T2w exists for 1023
All necessary data exists for 1023, making subject directory
Copying over T2w for 1023
Computing Mask for 1023
180620-16:11:18,515 interface INFO:
	 stdout 2018-06-20T16:11:18.514766:
180620-16:11:18,516 interface INFO:
	 stdout 2018-06-20T16:11:18.514766:$Id: mri_binarize.c,v 1.43 2016/06/09 20:46:21 greve Exp $
180620-16:11:18,516 interface INFO:
	 stdout 2018-06-20T16:11:18.514766:cwd /Volumes/iang/active/BABIES/BABIES_qT1/scripts
180620-16:11:18,517 interface INFO:
	 stdout 2018-06-20T16:11:18.514766:cmdline mri_binarize.bin --o /Volumes/iang/active/BABIES/BABIES_qT1/newproc/1023/1023-binarized_mask.nii --i /Volumes/iang/active/BABIES/BABIES_qT1/newproc/1023/1023_skullstripped.nii --inv --max 1.00

In [82]:
csubs = getsubs_c()

def copyqt1_c():
    qt1raw_pe1 = crawfp +'/'+sub+'-C-T1/anat/qt1/recon/qt1_pe1.nii.gz'
    qt1raw_pe0 = crawfp +'/'+sub+'-C-T1/anat/qt1/recon/qt1_pe0.nii.gz'
    subdir = newfp + '/'+ sub
    if os.path.exists(subdir):
        if os.path.exists(qt1raw_pe0):
            if os.path.exists(qt1raw_pe1):
                if os.path.exists(subdir+'/'+sub+'_qt1_raw_pe0.nii.gz'):
                    print('Raw qT1 data already copied over{}, skipping'.format(sub))
                else:
                    print('Raw qT1 data exists for {}, copying over...'.format(sub))
                    copyfile(qt1raw_pe0, subdir+'/'+sub+'_qt1_raw_pe0.nii.gz')
                    copyfile(qt1raw_pe1, subdir+'/'+sub+'_qt1_raw_pe1.nii.gz')
                    print('Data successfully copied for {}'.format(sub))
            else:
                print('*****qT1 pe1 data not found for {}*****'.format(sub))
        else:
            print('*****qT1 pe0 data not found for {}*****'.format(sub))
    else:
        print('Subject Directory does not exist for {}, no T2w'.format(sub))
        
for sub in csubs:
    copyqt1_c()

Subject Directory does not exist for 033x, no T2w
Subject Directory does not exist for 056, no T2w
*****qT1 pe0 data not found for 067*****
Subject Directory does not exist for 020, no T2w
Subject Directory does not exist for 028x, no T2w
Subject Directory does not exist for 122, no T2w
Subject Directory does not exist for 021, no T2w
Subject Directory does not exist for 010, no T2w
*****qT1 pe0 data not found for 072*****
Subject Directory does not exist for 039, no T2w
Raw qT1 data already copied over087, skipping
Subject Directory does not exist for 025, no T2w
Raw qT1 data already copied over123, skipping
Subject Directory does not exist for 031, no T2w
Raw qT1 data already copied over106, skipping
*****qT1 pe0 data not found for 062*****
Raw qT1 data already copied over064x, skipping
Subject Directory does not exist for 035, no T2w
Raw qT1 data already copied over102, skipping
Subject Directory does not exist for 023, no T2w
Subject Directory does not exist for 012, no T2w
Raw qT1

In [83]:
lsubs = getsubs_l()

def copyqt1_l():
    qt1raw_pe1 = lrawfp+'/'+sub+'-L-T3/anat/qt1/recon/qt1_pe1.nii.gz'
    qt1raw_pe0 = lrawfp+'/'+sub+'-L-T3/anat/qt1/recon/qt1_pe0.nii.gz'
    subdir = newfp + '/'+ sub
    if os.path.exists(subdir):
        if os.path.exists(qt1raw_pe0):
            if os.path.exists(qt1raw_pe1):
                if os.path.exists(subdir+'/'+sub+'_qt1_raw_pe0.nii.gz'):
                    print('Raw qT1 data already copied over for {}, skipping'.format(sub))
                else:
                    print('Raw qT1 data exists for {}, copying over...'.format(sub))
                    copyfile(qt1raw_pe0, subdir+'/'+sub+'_qt1_raw_pe0.nii.gz')
                    copyfile(qt1raw_pe1, subdir+'/'+sub+'_qt1_raw_pe1.nii.gz')
                    print('Data successfully copied for {}'.format(sub))
            else:
                print('*****qT1 pe1 data not found for {}*****'.format(sub))
        else:
            print('*****qT1 pe0 data not found for {}*****'.format(sub))
    else:
        print('Subject Directory does not exist for {}, no T2w'.format(sub))
for sub in lsubs:
    copyqt1_l()

Raw qT1 data already copied over for 1036, skipping
Raw qT1 data already copied over for 1027, skipping
Raw qT1 data exists for 1023, copying over...
Data successfully copied for 1023
Raw qT1 data exists for 1033, copying over...
Data successfully copied for 1033
Raw qT1 data already copied over for 1078, skipping
*****qT1 pe0 data not found for 1010*****
Raw qT1 data already copied over for 1029, skipping
*****qT1 pe0 data not found for 1005*****
Raw qT1 data already copied over for 1024, skipping
*****qT1 pe0 data not found for 1020*****
Raw qT1 data already copied over for 1011, skipping
